# API KEYS

In [1]:
import os
os.environ["GROQ_API_KEY"] = "Put You're GROQ API key here"
os.environ["TAVILY_API_KEY"] = "Put You're TAVILY API key here"

In [ ]:
! pip install -U tesseract langchain langchain_nomic langchain_groq langchain_community tiktoken langchain-openai langchain-cohere langchainhub chromadb langchain langgraph  tavily-python

# Data Embeddings

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_nomic.embeddings import NomicEmbeddings

# Initialize Nomic embeddings
embd = NomicEmbeddings(model="nomic-embed-text-v1.5")

pdf_folder_path = "./data/pdfs/"

# Load PDF files
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path) if fn.endswith('.pdf')]

all_documents = []

persist_dir = './vectorstore/'

for loader in loaders:
    raw_document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    docs = text_splitter.split_documents(raw_document)
    all_documents.extend(docs)  # Flatten the list of documents

vectorstore = Chroma.from_documents(
    documents=all_documents,
    collection_name="data_pdfs",
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    persist_directory=persist_dir
)
vectorstore.persist()

OllamaEmbeddings: 100%|██████████| 17/17 [00:06<00:00,  2.83it/s]


# LLM

### Query Analysis

In [ ]:
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

class RouteQuery(BaseModel):
    """Route a user query to the most relevant data source."""
    datasource: Literal["vectorstore", "web_search"] = Field(
        ..., description= "Given a user question choose to route it to web search or vectorstore.",
    )
llm = ChatGroq(model="llama3-70b-8192", temperature=0, api_key= os.environ["GROQ_API_KEY"])
structured_llm_router = llm.with_structured_output(RouteQuery)

system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to a company called omnidoc assist, it services, and general informations.
Use the vectorstore for questions on these topics. Otherwise, use web-search."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

### Translate Prompt

In [ ]:

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

translation_prompt = PromptTemplate(
    input_variables=["question"],
    template = """
You are an advanced translator. Your task is to accurately translate questions into English. 

Instructions:
- Translate the given question precisely.
- Respond only with the translation, without adding any extra information.

Question: {question}
Translation:
"""

)

translation_chain = translation_prompt | llm | StrOutputParser()

In [ ]:
translate_back_prompt = PromptTemplate(
    input_variables=["generation", "language"],
    template = """
You are an advanced translator. Your task is to accurately translate the text below into the {language} language.

Instructions:
- Translate the given text precisely.
- Respond only with the translation, without adding any extra information.

Text to translate: {generation}
Translation:
"""


)

translate_back_chain = translate_back_prompt | llm | StrOutputParser()

In [ ]:
lang_det_prompt = PromptTemplate(
    input_variables=["question"],
    template = """
You are an advanced language detector. Your task is to identify the language used in the question below.

Instructions:
- Detect the language of the given question.
- Respond with the language name in one word, without any explanation.

Question: {question}
Language:
"""

)

lang_det_chain = lang_det_prompt | llm | StrOutputParser()

### Rewrite question

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

rewrite_prompt = PromptTemplate(
input_variables=["question"],
template="""
You are an assistant designed to assist users in retrieving the most relevant documents from a vector database. 
Your task is to generate five distinct variations of the user's query to enhance the search results. 
These variations should capture different perspectives and nuances of the original question to mitigate the limitations of distance-based similarity searches. 
Please provide these alternative queries, each on a new line.

Original question: {question}

Variation 1:
Variation 2:
Variation 3:
Variation 4:
Variation 5:
"""
)

retriever = MultiQueryRetriever.from_llm(
    retriever= vectorstore.as_retriever(),
    llm = llm,
    prompt = rewrite_prompt
)

### Grader

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
    
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retriever_grader =grade_prompt | structured_llm_grader

### Generation

In [ ]:
from langchain import hub

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = prompt | llm | StrOutputParser()

### Hallucinations

In [ ]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

### Answer Grader

In [ ]:
### Answer Grader


# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader

# Web Search

In [ ]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

# Graph

In [ ]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    language: str
    generation: str
    documents: List[str]

In [ ]:
from langchain.schema import Document

def detect_language(state):
    """
    DETECT prompt language
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): get the prompt language
    """
    print("---DETECT---")
    question = state["question"]
    language = lang_det_chain.invoke(question)

    # Retrieval
    documents = state["documents"]
    return {"documents": documents, "question": question, "language": language}

def translate(state):
    """
    Translate prompt
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Question is changed
    """
    print("---TRANSLATE---")        
    question = state["question"]
    question = translation_chain.invoke(question)

    # Retrieval
    documents = state["documents"]
    return {"documents": documents, "question": question, "language": state["language"]}

def translate_back(state):
    """
    Translate prompt
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): generation is changed
    """
    print("---TRANSLATE BACK---")
    generation = translate_back_chain.invoke({"generation": state["generation"], "language": state["language"]})

    # Retrieval
    documents = state["documents"]
    question = state["question"]
    return {"documents": documents, "question": question, "generation": generation, "language": state["language"]}

def retrieve(state):
    """
    Retrieve documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question, "language": state["language"]}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation, "language": state["language"]}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retriever_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question, "language": state["language"]}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ###


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

### Build graph

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("translation", translate)  
workflow.add_node("translation_back", translate_back)
workflow.add_node("detect_language", detect_language)
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "web_search": "web_search",
        "vectorstore": "detect_language",
    },
)
workflow.add_edge("web_search", "generate")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("detect_language", "translation")
workflow.add_edge("translation", "retrieve")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "web_search": "web_search",
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": "translation_back",
        "not useful": "web_search",
    },
)
workflow.add_edge("translation_back", END)


# Compile
app = workflow.compile()


In [ ]:
# Run
inputs = {"question": "What are the 4 muscles in the chest?"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        print(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    print("\n---\n")

# Final generation
print(value["language"])
print(value["generation"])